In [1]:
#Importing Config and other Libraries
from pyspark.sql import SparkSession
import config
import numpy
from nsepy import get_history
from datetime import date,timedelta
import datetime
import os
import shutil
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType,DateType,FloatType,StructType,StructField,StringType

In [2]:
spark = SparkSession.builder.master('local').appName('stockupdate').enableHiveSupport().getOrCreate()
spark

23/02/27 16:18:18 WARN Utils: Your hostname, abhishek-HP-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.10.25 instead (on interface wlo1)
23/02/27 16:18:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/27 16:18:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option(key='header',value=True).csv(config.stock_list_path)
df.show()

+---------+
|   Stocks|
+---------+
|      ITC|
|      TCS|
|TATASTEEL|
|    TECHM|
+---------+



In [4]:
pdf = df.toPandas()
config.database

'nseproject'

In [5]:
schema = StructType([StructField(name='Date',dataType=DateType()),StructField(name='Symbol',dataType=StringType()),StructField(name='Prev Close',dataType=FloatType()),StructField(name='Open',dataType=FloatType()),StructField(name='High',dataType=FloatType()),StructField(name='Low',dataType=FloatType()),StructField(name='Close',dataType=FloatType())])

In [17]:
#Create table
for a in pdf.Stocks:
    query='create table if not exists ' + config.database + '.' + a + ' (date Date, stock String, pre_close Float, open Float, high Float, low Float, close Float) ROW FORMAT DELIMITED FIELDS TERMINATED BY ",";'
    spark.sql(query)


In [18]:
enddate = datetime.datetime.today()
startdate = enddate-datetime.timedelta(days=180)
for a in pdf.Stocks:
    temp_pd = get_history(symbol=a,start=startdate,end=enddate)
    temp_pd = temp_pd.drop(['Series','VWAP','Volume','Turnover','Trades','Deliverable Volume','%Deliverble'],axis=1)
    temp_pd.to_csv('/home/abhishek/test/nseproject/data_file.csv')
    temp_pd.head(5)
    #df_stock = spark.read.option(key='header',value=True).csv('/home/abhishek/test/nseproject/data_file.csv')
    #df_stock.show()
    #df_stock.coalesce(1).write.csv('hdfs://localhost:50000/nseproject/data_file.csv',mode='overwrite')
    truncate_query = 'TRUNCATE TABLE ' + config.database + '.' + a + ';'
    spark.sql(truncate_query)
    load_query='LOAD DATA LOCAL INPATH "/home/abhishek/test/nseproject/data_file.csv" INTO TABLE ' + config.database + '.' + a + ';'
    spark.sql(load_query)
